Создано: 28.12.2024

Автор: Лейман М.А.


# Декораторы

## Задача 1 Права администратора

In [11]:
# Определение глобальной переменной user_role
user_role = ""

def role_required(role: str):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if user_role == role:
                return func(*args, **kwargs)
            else:
                return "Permission denied"
        return wrapper
    return decorator

@role_required("admin")
def secret_resource() -> str:
    return "Permission accepted"


In [12]:
 #Пример использования
if __name__ == "__main__":
    print("\n")
    # Пример 1: Пользователь с ролью admin
    user_role = "admin"
    print(secret_resource())  # Ожидаемый вывод: Permission accepted
    print("\n")
    # Пример 2: Пользователь с ролью manager
    user_role = "manager"
    print(secret_resource())  # Ожидаемый вывод: Permission denied
    print("\n")



Permission accepted


Permission denied




## Задача 2 Кэширование

In [2]:
import sqlite3
import psycopg2
from psycopg2 import sql
from psycopg2.extras import RealDictCursor
from functools import wraps


In [ ]:

def cache(db: str, expiration: int):
    """
        Декоратор для кэширования с заданными параметрами
    """
    def decorator(func):
        cache_store = {}  # Хранилище кэша

        @wraps(func)
        def wrapper(thing):
            if thing in cache_store:
                cached_data, expire_time = cache_store[thing]

                if expire_time > 0:
                    cache_store[thing] = (cached_data, expire_time - 1)
                    return f"Info about: {thing} cached in {db}, expire={expire_time - 1}"
                else:
                    # Сброс кэша
                    del cache_store[thing]

            # Если данных в кэше нет или они были сброшены
            result = func(thing)
            cache_store[thing] = (result, expiration)
            return f"Info about: {thing} from {db}, now cached with expire={expiration}"

        return wrapper

    return decorator

# Функция для получения информации о предмете
def get_info(thing: str) -> str:
    return f"Info about: {thing}"

# Пример использования:
@cache(db="postgresql", expiration=5)
def get_postgresql_info(thing):
    return get_info(thing)

@cache(db="sqlite", expiration=3)
def get_sqlite_info(thing):
    return get_info(thing)

In [35]:

# Тестирование
if __name__ == "__main__":
    print(get_postgresql_info("bike_store"))
    print(get_postgresql_info("bike_store"))
    print(get_postgresql_info("bike_store"))
    print(get_postgresql_info("bike_store"))
    print(get_postgresql_info("bike_store"))
    print(get_postgresql_info("bike_store"))
    print(get_postgresql_info("bike_store"))

    print(get_sqlite_info("bike_store"))
    print(get_sqlite_info("bike_store"))
    print(get_sqlite_info("bike_store"))
    print(get_sqlite_info("bike_store"))
    print(get_sqlite_info("bike_store"))

Info about: bike_store cached in postgresql, expire=4
Info about: bike_store cached in postgresql, expire=3
Info about: bike_store cached in postgresql, expire=2
Info about: bike_store cached in postgresql, expire=1
Info about: bike_store cached in postgresql, expire=0
Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store cached in postgresql, expire=4
Info about: bike_store cached in sqlite, expire=0
Info about: bike_store from sqlite, now cached with expire=3
Info about: bike_store cached in sqlite, expire=2
Info about: bike_store cached in sqlite, expire=1
Info about: bike_store cached in sqlite, expire=0


### Хранение хеша в БД

In [4]:

def setup_database():
    """
        Создание Тестовой базы данных SQLite и с таблицей работников
    """
    conn = sqlite3.connect("example.db")
    cursor = conn.cursor()

    # Создание таблицы
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS items (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            description TEXT
        )
    """)

    # Заполнение таблицы данными
    items = [
        ("Иван", "Разработчик"),
        ("Пётр", "Администратор"),
        ("Михаил", "Директор"),
    ]
    cursor.executemany("INSERT INTO items (name, description) VALUES (?, ?)", items)

    conn.commit()
    conn.close()

setup_database()

In [ ]:

def create_postgresql_database(db_name: str, user: str, password: str, host: str, port: int):
    try:
        # Подключаемся к postgres, чтобы создать базу данных
        conn = psycopg2.connect(
            dbname=db_name,
            user=user,
            password=password,
            host=host,
            port=port
        )
        conn.autocommit = True  # Устанавливаем autocommit для создания базы
        cursor = conn.cursor()

        # Проверяем, существует ли база данных
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
        exists = cursor.fetchone()

        if not exists:
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
            print(f"БД  '{db_name}' Создана.")
        else:
            print(f"БД '{db_name}' уже существует")

        cursor.close()
        conn.close()
    except psycopg2.Error as e:
        print(f"Ошибка создания БД PostgreSQL: {e}")



# Создание базы данных PostgreSQL и заполнение таблицы
def setup_postgresql_database():
    """
        Cоздание базы данных PostgreSQL и заполнение таблицы 
    """
    # Учетные данные для подключения
    user = "postgres"
    password = "postgres"
    host = "localhost"
    port = 5432
    db_name = "db1"

    # Создаем базу данных, если она отсутствует
    create_postgresql_database(db_name, user, password, host, port)

    import time
    time.sleep(5)

    # Подключаемся к созданной базе
    conn = psycopg2.connect(
        dbname=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cursor = conn.cursor()

    # Создание таблицы кэша
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cache (
            id SERIAL PRIMARY KEY,
            db TEXT NOT NULL,
            name TEXT NOT NULL,
            description TEXT,
            expire_time INTEGER
        )
    """)
    
    conn.commit()
    cursor.close()
    conn.close()
    print(f"БД '{db_name}' настроена")

setup_postgresql_database()

БД 'db1' уже существует
БД 'db1' настроена


In [7]:

def cache(db: str, expiration: int):
    """
        Декоратор для кэширования
    """
    def decorator(func):
        @wraps(func)
        def wrapper(thing):
            conn = psycopg2.connect(
                dbname="db1",
                user="postgres",
                password="postgres",
                host="localhost",
                port=5432
            )
            conn.autocommit = True
            cursor = conn.cursor()

            # Проверка наличия записи в кэше
            cursor.execute("""
                SELECT description, expire_time FROM cache
                WHERE db = %s AND name = %s
            """, (db, thing))
            row = cursor.fetchone()

            if row:
                cached_data, expire_time = row

                if expire_time > 0:
                    # Уменьшение времени жизни кэша
                    cursor.execute("""
                        UPDATE cache
                        SET expire_time = %s
                        WHERE db = %s AND name = %s
                    """, (expire_time - 1, db, thing))
                    conn.commit()
                    conn.close()
                    return f"Информация о: {thing} кэшировано в {db}, expire={expire_time - 1}"

                # Удаление записи при истечении срока действия
                cursor.execute("""
                    DELETE FROM cache
                    WHERE db = %s AND name = %s
                """, (db, thing))

            # Если данных нет в кэше или срок действия истёк
            result = func(thing)

            # Сохранение данных в кэш
            cursor.execute("""
                INSERT INTO cache (db, name, description, expire_time)
                VALUES (%s, %s, %s, %s)
            """, (db, thing, result, expiration))

            conn.commit()
            conn.close()
            return f"Информация о: {thing} из  {db}, кэширована  с expire={expiration}"

        return wrapper

    return decorator


In [ ]:

def get_info(thing: str) -> str:
    ''' 
        Заглушка: реальная информация могла бы приходить из БД или API
    '''
    info = {
        "bikeStore": "Магазин, торгующий велосипедами",
        "users": "Информация о пользователях",
        "library": "Библиотека",
        "db1": "Хэширование БД",
        "example.db": "Тестовая БД с таблицей работников"
    }

    # Нормализуем входной параметр, чтобы избежать ошибок из-за регистра
    normalized_thing = thing.strip()
    
    # Проверяем, есть ли информация для переданного ключа
    result = info.get(normalized_thing, f"Информация не найдена для {normalized_thing}")
    return result


# Пример использования декоратора для SQLite и PostgreSQL
@cache(db="sqlite", expiration=6)
def get_sqlite_info(thing):
    return get_info(thing)

@cache(db="postgresql", expiration=4)
def get_postgresql_info(thing):
    return get_info(thing)

# Тестирование
if __name__ == "__main__":
    # # Подготовка баз данных
    # setup_sqlite()
    # setup_postgresql()

    # Тестирование SQLite
    print(get_sqlite_info("users"))
    print(get_sqlite_info("bikeStore"))
    print(get_sqlite_info("library"))
    print(get_sqlite_info("example.db"))  

    # Тестирование PostgreSQL
    print(get_postgresql_info("db1"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))  


Информация о: users из  sqlite, кэширована  с expire=6
Информация о: bikeStore из  sqlite, кэширована  с expire=6
Информация о: library из  sqlite, кэширована  с expire=6
Информация о: example.db из  sqlite, кэширована  с expire=6
Информация о: db1 из  postgresql, кэширована  с expire=4
Информация о: library из  postgresql, кэширована  с expire=4


In [13]:
# Тестирование
if __name__ == "__main__":
    # # Подготовка баз данных
    # setup_sqlite()
    # setup_postgresql()

    # Тестирование SQLite
    print(get_sqlite_info("users"))
    print(get_sqlite_info("bikeStore"))
    print(get_sqlite_info("library"))
    print(get_sqlite_info("example.db"))  

    # Тестирование PostgreSQL
    print(get_postgresql_info("db1"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))
    print(get_postgresql_info("library"))  
    print(get_postgresql_info("library"))

Информация о: users кэшировано в sqlite, expire=1
Информация о: bikeStore кэшировано в sqlite, expire=1
Информация о: library кэшировано в sqlite, expire=1
Информация о: example.db кэшировано в sqlite, expire=1
Информация о: db1 из  postgresql, кэширована  с expire=4
Информация о: library кэшировано в postgresql, expire=1
Информация о: library кэшировано в postgresql, expire=0
Информация о: library из  postgresql, кэширована  с expire=4
Информация о: library кэшировано в postgresql, expire=3
Информация о: library кэшировано в postgresql, expire=2
Информация о: library кэшировано в postgresql, expire=1


## Задача 3 Контекстный менеджер safe_write

In [56]:
class safe_write:
    def __init__(self, filename):
        self.filename = filename
        self.original_data = None

    def __enter__(self):
        # Сохраняем оригинальное содержимое файла
        try:
            with open(self.filename, 'r') as file:
                self.original_data = file.read()
        except FileNotFoundError:
            self.original_data = None  # Если файла нет, это не проблема

        # Открываем файл в режиме записи
        self.file = open(self.filename, 'w')
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        self.file.close()  # Закрываем файл
        if exc_type is not None:  # Если возникло исключение
            # Восстанавливаем оригинальное содержимое
            with open(self.filename, 'w') as file:
                if self.original_data is not None:
                    file.write(self.original_data)
            print(f"Во время записи в файл было возбуждено исключение {exc_type.__name__}")
            return True  # Подавляем исключение


In [57]:
with safe_write('Файл.txt') as file:
    file.write('Я знаю, что ничего не знаю, но другие не знают и этого. \n')
    
with open('Файл.txt') as file:
    print(file.read())   

Я знаю, что ничего не знаю, но другие не знают и этого. 



In [58]:
with safe_write('Файл.txt') as file:
    print(
        'Если ты будешь любознательным, то будешь много знающим.',
        file=file,
        flush=True
    )
    raise ValueError

with open('Файл.txt') as file:
    print(file.read())


Во время записи в файл было возбуждено исключение ValueError
Я знаю, что ничего не знаю, но другие не знают и этого. 

